### This code extracts documents and related comments from documents in different categories. There are 10 categories in regulation.gov

In [1]:
from pickle import dump, load
import pandas
import requests
import urllib.request
from bs4 import BeautifulSoup
import PyPDF2
api_key = 'b5Uc6UQwdVYBhNV20O11AZFc6s2cMZ8tpYrUc9tV' # has increased rate of 2500 -- 1247 comments at a time

In [4]:
# def read_file_get_docid(filepath):
#     dump_df = load(open(filepath,'rb'))
#     df_with_comments = dump_df[dump_df.numberOfCommentsReceived > 0]
#     df_with_comments =df_with_comments.sort(['numberOfCommentsReceived'], ascending=[False])
#     doc_id = df_with_comments.documentId
#     doc_type = df_with_comments.documentType
#     #return [doc_id,set(doc_type)]
#     return df_with_comments

In [16]:
# [doc_id_list,types] = read_file_get_docid('data/AD_doc_list') #choose the category dump file
# print(types)
# # document ID with 4 parts represent documents. 3 parts represent dockets 
# doc_ids = [doc_id for doc_id in doc_id_list if len(doc_id.split('-')) == 4]
# len(doc_ids)

{'Proposed Rule', 'Rule', 'Other', 'Notice'}


/Users/kinshuk/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


4421

### Using regulations.gov API
* We need to use the API to retrieve each document content. This API will use document_id that we extracted from the file above.
* For each document_id, we will need to construct comment_id based on the total number of comments on it.

In [2]:
def download_file(download_url):
    try:
        response = urllib.request.urlopen(download_url)
        file = open("document.pdf", 'wb')
        file.write(response.read())
        file.close()
    except:
        print("(downloading the pdf exception)error log" +  download_url)
    
def get_attached_comments(comment_id, key=api_key):
    #print(each_id) # fro debugging
    #open the api to get file url
    url = "http://api.data.gov:80/regulations/v3/document.json?api_key="+key+"&documentId="+comment_id
    try:
        response = requests.get(url)
    except:
        print("(api opening of attached comment exception)error log" +  url)
    if response.status_code != 200:
        print("status code " +str(response.status_code)+" (get_attached_comments) program will break at this point which is ok because we dont need inconsistent data. Run again ")
    data = response.json()
    att_count = 0
    for i in range(len(data["attachments"])):
        if "fileFormats" in data["attachments"][i]:
            att_count = len(data["attachments"][0]["fileFormats"])
    comment_text =""
    for i in range(att_count):
        if data["attachments"][0]["fileFormats"][i].endswith("pdf"):
            link = data["attachments"][0]["fileFormats"][i] 
            access_link = link+'&api_key='+key
            #download file(pdf) and read pdf (page by page)
            download_file(access_link)
            pdfFileObj = open('document.pdf','rb')     #'rb' for read binary mode
            try:
                pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
                pno = pdfReader.numPages
                for i in range(pno):
                    pageObj = pdfReader.getPage(i)          #'i' is the page number
                    comment_text += pageObj.extractText()
            except:
                print("(pdf exception)cant read "+comment_id ) # prints in case we are not able to read file
            break # execute the whole thing for 1st found pdf
    return comment_text

In [3]:
def get_document_comments_from_api(docketId,key=api_key):
    offset=0
    url = "http://api.data.gov:80/regulations/v3/documents.json?api_key="+key+"&countsOnly=1&dct=PS&dktid="+docketId
    try:
        response = requests.get(url)
    except:
        print("(api opening comment count)error log"+url) # prints in case we are not able to read file
    if response.status_code != 200:
        print("status code"+str(response.status_code) + " (get_document_comments_from_api) program will break at this point which is ok because we dont need inconsistent data. Run again")
    data = response.json()
    total = data['totalNumRecords']
    com_list =[]
    for i in range(0,total,1000):
        url = "http://api.data.gov:80/regulations/v3/documents.json?api_key="+key+"&countsOnly=0&&rpp=1000&po="+str(i)+"&dct=PS&dktid="+docketId
        try:
            response = requests.get(url)
        except:
            print("(api opening actual comments)error log"+url) # prints in case we are not able to read file
        #print("Offset:"+str(i)+" Code:"+str(response.status_code))
        if response.status_code != 200:
            print(response.status_code)
        data = response.json()
        com_list += data['documents']
    com_df = pandas.DataFrame(com_list)
    return com_df

def get_document_content_from_api(docId,key=api_key):
    url = "http://api.data.gov:80/regulations/v3/document.json?api_key="+key+"&documentId="+docId
    try:
        response = requests.get(url)
    except:
         print("(api opening doc exception) error log"+url)   
    if response.status_code != 200:
        print("status code "+str(response.status_code)+" (get_document_content_from_api) program will break at this point which is ok because we dont need inconsistent data. Run again")
    data = response.json()
    
    # Get HTML for document content
    if(len(data['fileFormats']) == 2):    
        link = data['fileFormats'][1] # The second link is the document in HTML format
    else:
        link = data['fileFormats'][0]
    access_link = link+'&api_key='+key
    
    try:
        with urllib.request.urlopen(access_link) as response:
            html = response.read()
    except:
        print("doc file opening exception")
    
    # We are interested in the pre tag of the HTML content
    soup = BeautifulSoup(html, "lxml")
    content = soup.find_all('pre')
    
    # Now we need to construct comment_id from document_id
    docket_id = '-'.join(docId.split('-')[:3])
    comment_df = get_document_comments_from_api(docket_id)
    # get comment text where exists
    comment_list =[]
    if not comment_df.empty:
        if "commentText" in comment_df:
            comment_text =comment_df[comment_df.commentText.notnull()].commentText
            comment_list =comment_text.tolist()
        #get doc id where there is attchment
        c_ids = comment_df[comment_df.attachmentCount>0].documentId
        # get comment for each id in list
        for each_id in c_ids.unique():
            comment_list.append(get_attached_comments(each_id))
    doc_dict = {
        "text":content,
        "comment_list":comment_list
    }
    return doc_dict

#### Running it on one document

In [14]:
# data format is array of dicts
def get_one_doc(name,docid):
    #open file - get present data (empty to begin witj)
    filepath = "data/"+name+"_doc_content"
    inp =open(filepath,'rb')
    doc_collection = load(inp)
    inp.close()
    #get that one doc
    #resp = get_document_content_from_api(docid)
    doc_collection.append(docid)
    #put back in file 
    output = open(filepath, 'wb')
    dump(doc_collection, output, -1)
    output.close()


In [31]:
# run this line once every 1 hr
get_one_doc("Emily",one_dc)

RecursionError: maximum recursion depth exceeded

# BELOW THIS IS TESTING, NOT NEEDED TO GET DOCUMENT AND COMMENTS CURRENTLY

In [32]:
one_dc=get_document_content_from_api('CFPB-2013-0033-0001')

(pdf exception)cant read CFPB-2013-0033-0347


(pdf exception)cant read CFPB-2013-0033-0068


(pdf exception)cant read CFPB-2013-0033-0373


In [30]:
one_dc['text']

[<pre>
 [Federal Register Volume 79, Number 168 (Friday, August 29, 2014)]
 [Proposed Rules]
 [Pages 51731-51881]
 From the Federal Register Online via the Government Printing Office [<a href="http://www.gpo.gov">www.gpo.gov</a>]
 [FR Doc No: 2014-18353]
 
 
 
 [[Page 51731]]
 
 Vol. 79
 
 Friday,
 
 No. 168
 
 August 29, 2014
 
 Part IV
 
 
 
 
 
  Bureau of Consumer Financial Protection
 
 
 
 
 
 -----------------------------------------------------------------------
 
 
 
 
 
 12 CFR Part 1003
 
 
 
 
 
  Home Mortgage Disclosure (Regulation C); Proposed Rule
 
   Federal Register / Vol. 79 , No. 168 / Friday, August 29, 2014 / 
 Proposed Rules  
 
 [[Page 51732]]
 
 
 -----------------------------------------------------------------------
 
 BUREAU OF CONSUMER FINANCIAL PROTECTION
 
 12 CFR Part 1003
 
 [Docket No. CFPB-2014-0019]
 RIN 3170-AA10
 
 
 Home Mortgage Disclosure (Regulation C)
 
 AGENCY: Bureau of Consumer Financial Protection.
 
 ACTION: Proposed rule with request fo

In [26]:
# will return the number of documents read. replace by your name
dc =load(open("data/Emily_doc_content",'rb'))
len(dc)

9

In [27]:
dc[8]["text"]

[<pre>
 [Federal Register Volume 81, Number 86 (Wednesday, May 4, 2016)]
 [Proposed Rules]
 [Pages 26903-26940]
 From the Federal Register Online via the Government Publishing Office [<a href="http://www.gpo.gov">www.gpo.gov</a>]
 [FR Doc No: 2016-10297]
 
 
 
 [[Page 26903]]
 
 Vol. 81
 
 Wednesday,
 
 No. 86
 
 May 4, 2016
 
 Part III
 
 
 
 
 
 Department of Homeland Security
 
 
 
 
 
 -----------------------------------------------------------------------
 
 
 
 
 
 8 CFR Parts 103 and 204
 
 
 
 
 
 U.S. Citizenship and Immigration Services Fee Schedule; Proposed Rule
 
   Federal Register / Vol. 81 , No. 86 / Wednesday, May 4, 2016 / 
 Proposed Rules  
 
 [[Page 26904]]
 
 
 -----------------------------------------------------------------------
 
 DEPARTMENT OF HOMELAND SECURITY
 
 8 CFR Parts 103 and 204
 
 [CIS No. 2577-15; DHS Docket No. USCIS-2016-0001]
 RIN 1615-AC09
 
 
 U.S. Citizenship and Immigration Services Fee Schedule
 
 AGENCY: U.S. Citizenship and Immigration Ser

In [3]:
df_test =read_file_get_docid('data/BFS_doc_list')
df_test.reset_index(inplace = True)

/Users/kinshuk/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [8]:
df_test["numberOfCommentsReceived"][20:65] #60,61,62,63

20    1689
21    1624
22    1473
23    1440
24    1425
25    1165
26     998
27     979
28     972
29     854
30     843
31     839
32     829
33     741
34     671
35     622
36     592
37     555
38     539
39     538
40     425
41     408
42     406
43     398
44     367
45     320
46     318
47     298
48     291
49     283
50     252
51     231
52     230
53     224
54     224
55     218
56     216
57     212
58     202
59     201
60     193
61     182
62     181
63     166
64     165
Name: numberOfCommentsReceived, dtype: int64

In [9]:
df_test["documentId"][40:50]

40      CFPB-2014-0019-0001
41      EBSA-2012-0031-0001
42       OCC-2011-0002-0001
43      CFPB-2013-0033-0001
44      CFPB-2012-0039-0001
45    FINCEN-2015-0002-0001
46       IRS-2012-0009-0001
47       IRS-2010-0009-0001
48      CFPB-2013-0018-0001
49       SBA-2007-0009-0002
Name: documentId, dtype: object